In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollator, T5ForConditionalGeneration, T5TokenizerFast

from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)

import numpy as np
import torch

In [56]:
df = pd.read_csv('cleaned_data.csv')

In [57]:
df.head()

,question,sql
0,Tell me what the notes are for South Australia,SELECT Notes FROM table WHERE Current slogan =...
1,What is the current series where the new serie...,SELECT Current series FROM table WHERE Notes =...
2,What is the format for South Australia?,SELECT Format FROM table WHERE State/territory...
3,Name the background colour for the Australian ...,SELECT Text/background colour FROM table WHERE...
4,how many times is the fuel propulsion is cng?,SELECT COUNT Fleet Series (Quantity) FROM tabl...


In [58]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments

In [59]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

C:\Users\MSI1\AppData\Roaming\Python\Python39\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Preprocess the dataset
def preprocess_data(row):
    input_text =row['question']
    target_text = row['sql']
    
    # Tokenize and encode the input and target text
    input_ids = tokenizer.encode(input_text, truncation=True, padding='max_length', max_length=128)
    target_ids = tokenizer.encode(target_text, truncation=True, padding='max_length', max_length=128)
    
    return {
        'input_ids': input_ids,
        'attention_mask': [1] * len(input_ids),
        'target_ids': target_ids,
        'target_attention_mask': [1] * len(target_ids)
    }

preprocessed_data = df.apply(preprocess_data, axis=1)

In [ ]:
preprocessed_data

In [ ]:
train_data = preprocessed_data[:40000]
valid_data = preprocessed_data[40000:45000]
test_data = preprocessed_data[45000:]

In [ ]:
train_data

In [47]:
train_dataset = {
    'input_ids': torch.tensor(list(train_data.apply(lambda x: x['input_ids']))),
    'attention_mask': torch.tensor(list(train_data.apply(lambda x: x['attention_mask']))),
    'target_ids': torch.tensor(list(train_data.apply(lambda x: x['target_ids']))),
    'target_attention_mask': torch.tensor(list(train_data.apply(lambda x: x['target_attention_mask'])))
}

valid_dataset = {
    'input_ids': torch.tensor(list(valid_data.apply(lambda x: x['input_ids']))),
    'attention_mask': torch.tensor(list(valid_data.apply(lambda x: x['attention_mask']))),
    'target_ids': torch.tensor(list(valid_data.apply(lambda x: x['target_ids']))),
    'target_attention_mask': torch.tensor(list(valid_data.apply(lambda x: x['target_attention_mask'])))
}

test_dataset = {
    'input_ids': torch.tensor(list(test_data.apply(lambda x: x['input_ids']))),
    'attention_mask': torch.tensor(list(test_data.apply(lambda x: x['attention_mask']))),
    'target_ids': torch.tensor(list(test_data.apply(lambda x: x['target_ids']))),
    'target_attention_mask': torch.tensor(list(test_data.apply(lambda x: x['target_attention_mask'])))
}

In [54]:
train_dataset

In [49]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset

# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Define the T5 model configuration
model = T5ForConditionalGeneration.from_pretrained('t5-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model.to(device)

C:\Users\MSI1\AppData\Roaming\Python\Python39\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [52]:
batch_size = 4
num_epochs = 3
learning_rate = 1e-4

In [53]:
train_dataset

In [35]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    weight_decay=0.01,
    logging_steps=500,
    save_steps=1000,
    evaluation_strategy='steps',
    eval_steps=1000,
    save_total_limit=3,
)

In [36]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=None,
    tokenizer=tokenizer,
)

IndexError: too many indices for tensor of dimension 2

In [27]:
trainer.train()

C:\Users\MSI1\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyError: 0